<a href="https://colab.research.google.com/github/vadim13213/neural_networks/blob/main/5.%20%D0%90%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D1%83%D1%80%D1%8B%20%D0%A1%D0%9D%D0%A1%20%D0%B8%20Transfer%20Learning/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%965_%D0%90%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D1%83%D1%80%D1%8B_%D0%9D%D0%A1_%D0%B8_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1. Использование предобученной модели

### 1.1. Выберите и импортируйте предобученную модель СНС для решения задач классификации из имеющихся в Keras: https://keras.io/api/applications/

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from google.colab.patches import cv2_imshow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
model = keras.applications.ResNet50V2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
    name="resnet50v2",
)

model.summary()

### 1.2. Загрузите из сети Интернет 5 изображений с разными классами, из тех, что приведены в [списке](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a), и сохраните их представление в список images

In [ ]:
images = []

img_1 = cv2.imread('/content/agama.jpg')
img_1 = cv2.resize(img_1, (320, 320))

img_2 = cv2.imread('/content/brain coral.jpg')
img_2 = cv2.resize(img_2, (320, 320))

img_3 = cv2.imread('/content/hen.jpg')
img_3 = cv2.resize(img_3, (320, 320))

img_4 = cv2.imread('/content/toy terrier.jpg')
img_4 = cv2.resize(img_4, (320, 320))

img_5 = cv2.imread('/content/weevil.JPG')
img_5 = cv2.resize(img_5, (320, 320))

images.append(img_1)
images.append(img_2)
images.append(img_3)
images.append(img_4)
images.append(img_5)
images = np.array(images)

images.shape

### 1.3. Напишите функцию, принимающую на вход список с исходными изображениями и возвращающую список с преобразованными изображениями под формат входных данных выбранной Вами модели СНС (нормализация).

(Не забудьте создать независимую копию исходного списка в теле функции, для обработки именно независимой копии, а не оригинального списка)

In [ ]:
def normalize(images, shape):
  proc_images = []
  for image in images:
    proc_image = cv2.resize(image, (shape[0], shape[1]))
    proc_image = proc_image.reshape(shape[0], shape[1], shape[2])
    proc_images.append(proc_image)
  proc_images = np.array(proc_images)

  return proc_images

input_images = normalize(images, (224, 224, 3))

input_images = keras.applications.resnet_v2.preprocess_input(input_images)

input_images.shape

### 1.5. Выведите изображения и соответствующие им классы (как в примере из теории)

In [ ]:
class_name = ['agama', 'brain coral', 'hen', 'toy terrier', 'weevil']

plt.figure(figsize=(8, 8))
for i in range(5):
  image = images[i]
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image[:, :, ::-1])
  plt.title(class_name[i])
  plt.axis("off")

In [ ]:

res = model.predict(input_images)
for i in res:
  print(np.argmax(i))

In [ ]:
res.shape

In [ ]:
true_res = keras.applications.resnet_v2.decode_predictions(res, top=3)
for i in range(5):
  print(true_res[i])

# Задание 2. Использование обученной модели для работы со своим датасетом

### 2.1. Выберите и импортируйте предобученную модель СНС для решения задач классификации из имеющихся в Keras: https://keras.io/api/applications/

In [ ]:
basic_model = keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=(128, 128, 3),
    pooling='avg',
)

basic_model.summary()

### 2.2. Подготовка тренировочных данных. Разделение на обучающую и тестовую выборку

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
'''
from imutils import paths
import random

imagesPath = sorted(list(paths.list_images('/content/animals/')))
random.shuffle(imagesPath)
'''

import pathlib

imagesPath = pathlib.Path('/content/drive/MyDrive/Test/training_set')

In [ ]:
img_h, img_w = 128, 128
batch_size = 54

def preprocess(ds):
    return ds.map(lambda x, y: (keras.applications.resnet_v2.preprocess_input(x), y))

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    imagesPath,
    validation_split=0.2,
    subset='training',
    seed=420,
    image_size=(img_h, img_w),
    batch_size=batch_size
)
class_names = train_ds.class_names

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    imagesPath,
    validation_split=0.2,
    subset='validation',
    seed=420,
    image_size=(img_h, img_w),
    batch_size=batch_size
)

train_ds = preprocess(train_ds)
val_ds = preprocess(val_ds)

In [ ]:
print(class_names)

### 2.3. Обучение своей модели на основе предобученной модели "*название модели, выбранной в п.2.1.*"

In [ ]:
for layer in basic_model.layers:
  layer.trainable = False

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Input

input_tensor = basic_model.input
x = basic_model.output

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
output_tensor = Dense(3, activation='softmax')(x)

my_new_model = Model(inputs=input_tensor, outputs=output_tensor)

my_new_model.summary()

In [ ]:
my_new_model.compile(optimizer=Adam(learning_rate=0.001),
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(patience=5)
checkpoint = ModelCheckpoint(filepath="resnet50v2_best.keras", monitor="val_accuracy", save_best_only=True, verbose=0)

In [ ]:
history = my_new_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stopping, checkpoint]
)

### 2.4. Визуализация процесса обучения модели

In [ ]:
plt.figure(figsize=(20, 6))

ax = plt.subplot(1,2,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])

ax = plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])

plt.show()

### 2.5. Подготовьте набор проверочных данных (examples)

In [ ]:
# Ваш код

### 2.6. Реализуйте графический интерфейс для взаимодействия с обученной моделью, используя [Gradio](https://www.gradio.app/guides/image-classification-in-tensorflow). В качестве примеров (параметр examples в Gradio), используйте изображения из списка images (п.1.2.)

In [ ]:
# Ваш код

# Задание 3. Классификация набора пространственных данных EuroSat (для магистров)

- Ссылка на датасет: https://github.com/phelber/eurosat

- Описание на kaggle: https://www.kaggle.com/datasets/apollo2506/eurosat-dataset

В этом задании Вам будет предоставлен набор данных EuroSat, который содержит снимки земной поверхности, снятые спутником Sentinel-2. Вам нужно будет обучить модель для классификации этих изображений на 10 классов, представляющих различные типы земного покрова.

### **3.1. Загрузка данных:**
  * [Скачайте](https://madm.dfki.de/files/sentinel/EuroSAT.zip) набор данных EuroSat
  * Разделите данные на обучающую, валидационную и тестовую выборки.



In [ ]:
# Ваш код

### **3.2. Выбор архитектуры нейронной сети:**
  * Выберите архитектуру нейронной сети, основанную на трансферном обучении, например, ResNet50, VGG16 или InceptionV3 ([полный список](https://keras.io/api/applications/)).
  * Загрузите предобученную модель выбранной архитектуры.


In [ ]:
# Ваш код

### **3.3. Дообучение модели:**
  * Добавьте к предобученной модели несколько слоев для классификации изображений EuroSat.
  * Обучите модель на обучающей выборке.

In [ ]:
# Ваш код



### **3.4. Оценка модели:**
  * Оцените производительность модели на валидационной и тестовой выборках.
  * Сравните результаты с другими архитектурами нейронных сетей.



In [ ]:
# Ваш код

### **3.5. Анализ результатов:**
  * Проанализируйте влияние различных параметров на производительность модели, таких как:
      * Количество эпох обучения
      * Размер пакета
      * Функция активации
      * Оптимизатор
      * Регуляризация

-----------------------------------------
**Ваш вывод**
-----------------------------------------
